In [ ]:
import pandas as pd
import datetime as dt
import os
import glob

In [ ]:
path = r'../data/optimization'
os.chdir(path)
hist_list = glob.glob('*.csv')
# df_fx = pd.read_csv(path+ "/" + fx_csv)

In [ ]:
# Parametros de entrada
metrics_list = ['METRIC', 'PROFIT', 'TOTAL TRADES', 'PROFIT FACTOR', 'EXPECTED PAYOFF','EQUITY DD MAX', 'EFICIENCIA CURVA', 
                'TRADES PER MONTH', 'PROFIT % PER YEAR', 'CA', 'DW%', 'RELACION 24 MESES', 'RELACION 12 MESES', 
                'RELACION 6 MESES', 'MEDIA CO', 'MAXIMO CO']
date_start = '2015-01-01'
date_end = '2019-12-31'
anual_return = 0.1
max_dd_allow = 0.12
min_trades_month = 2.5

# Caculating filters
date_start = dt.datetime.strptime(date_start, '%Y-%m-%d')
date_end = dt.datetime.strptime(date_end, '%Y-%m-%d')
total_months = (date_end.year - date_start.year) * 12 + (date_end.month - date_start.month) +1
print('Total months: {}'.format(total_months))
np_dd_ratio = round((anual_return/12*total_months)/max_dd_allow,2)
print('Ratio Net Profit / DD allowed: {}'.format(np_dd_ratio))
total_trades = round(min_trades_month*total_months,0)
print('Min trades: {}'.format(total_trades))

In [ ]:
df_opt = pd.read_csv(hist_list[0], sep = ';').set_index('#')
df_opt

In [ ]:
print('Original rows: {}'.format(len(df_opt)))

In [ ]:
df_param = df_opt.drop(metrics_list, axis = 1)

# Delete repeated params
df_param.drop_duplicates(inplace = True)
df_param

In [ ]:
df_opt = df_opt.loc[df_param.index]
df_opt.reset_index(drop = True, inplace = True)
df_opt

In [ ]:
df_metrics = df_opt[metrics_list]

# Cleaning opt
df_metrics = df_metrics[df_metrics['PROFIT FACTOR'] >= 1.3]
df_metrics = df_metrics[df_metrics['RELACION 6 MESES'] >= 0.0]
df_metrics = df_metrics[df_metrics['RELACION 12 MESES'] >= 0.25]
df_metrics = df_metrics[df_metrics['RELACION 24 MESES'] >= 0.5]
df_metrics = df_metrics[(df_metrics['PROFIT'] /  df_metrics['EQUITY DD MAX']) >= np_dd_ratio]
df_metrics = df_metrics[df_metrics['TOTAL TRADES'] >= total_trades]

df_metrics

In [ ]:
df_opt = df_opt.loc[df_metrics.index]
df_opt.reset_index(drop = True, inplace = True)
df_opt

In [ ]:
from sklearn.cluster import AffinityPropagation

df_param = df_opt.drop(metrics_list, axis = 1)
df_copy = df_param.iloc[:10000].copy()

# Fit AFfinity Propagation with Scikit
afprop = AffinityPropagation(max_iter=500, random_state=5, )
af = afprop.fit(df_copy)

In [ ]:
centers = af.cluster_centers_indices_

In [ ]:
df_copy.iloc[centers]